# Playing Around with Model Selection Small Transformer

In [1]:
import numpy as np
import torch
import torch.nn as nn
from data_gen import get_data
from models import TransformerModel

N=21
d_d=5

In [2]:
data = get_data(N=21, d_d=5)

In [ ]:
def train_step(model, xs, ys, optimizer, loss_func):
    optimizer.zero_grad()
    output = model(xs, ys)
    loss = loss_func(output, ys)
    loss.backward()
    optimizer.step()
    return loss.detach().item(), output.detach()

def sample_seeds(total_seeds, count):
    seeds = set()
    while len(seeds) < count:
        seeds.add(randint(0, total_seeds - 1))
    return seeds

def train(model):
    optimizer = torch.optim.Adam(model.parameters())

    starting_step = 0
    # state_path = os.path.join(args.out_dir, "state.pt")
    # if os.path.exists(state_path):
    #     state = torch.load(state_path)
    #     model.load_state_dict(state["model_state_dict"])
    #     optimizer.load_state_dict(state["optimizer_state_dict"])
    #     starting_step = state["train_step"]
    #     for i in range(state["train_step"] + 1):
    #         curriculum.update()

    # n_dims = model.n_dims
    bsize = 32
    num_training_examples = 20000

    # task = task_sampler(**task_sampler_args)
    xs = data_sampler.sample_xs(curriculum.n_points, bsize, curriculum.n_dims_truncated, **data_sampler_args)
    ys = task.evaluate(xs)

    # Use MSE loss
    loss_func = nn.MSELoss()

    loss, output = train_step(model, xs.to(device), ys.to(device), optimizer, loss_func)

    point_wise_tags = list(range(curriculum.n_points))
    point_wise_loss_func = task.get_metric()
    point_wise_loss = point_wise_loss_func(output, ys.to(device)).mean(dim=0)
    if model.n_out > 1:
        point_wise_loss = point_wise_loss.mean(dim=-1)

    baseline_loss = (sum(max(curriculum.n_dims_truncated - ii, 0) for ii in range(curriculum.n_points)) / curriculum.n_points)

    if i % args.wandb.log_every_steps == 0 and not args.test_run:
        wandb.log(
            {
                "overall_loss": loss,
                "excess_loss": loss / baseline_loss,
                "pointwise/loss": dict(
                    zip(point_wise_tags, point_wise_loss.cpu().numpy())
                ),
                "n_points": curriculum.n_points,
                "n_dims": curriculum.n_dims_truncated,
                "n_embd": args.model.n_embd
            },
            step=i,
        )

    curriculum.update()

    pbar.set_description(f"loss {loss}")
    if i % args.training.save_every_steps == 0 and not args.test_run:
        training_state = {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_step": i,
        }
        torch.save(training_state, state_path)

    if (
        args.training.keep_every_steps > 0
        and i % args.training.keep_every_steps == 0
        and not args.test_run
        and i > 0
    ):
        torch.save(model.state_dict(), os.path.join(args.out_dir, f"model_{i}.pt"))

In [ ]:
model = TransformerModel(
    n_dims=d_d,
    n_positions=N
)

train(model)